In [9]:
import csv
import os
import datetime as datetime
from datetime import datetime, timezone

In [16]:
user="TylerTwohig"

user_input = """
Hello my name is Tyler Twohig. 
I am a master's student in Information Management & Systems 
at the University of California, Berkeley
"""

model = "DeepSeek-R1:latest"

In [11]:
def get_iso_time():
    now = datetime.now(timezone.utc)
    timestamp_iso = now.isoformat(timespec="milliseconds")
    timestamp_iso = timestamp_iso.replace("+00:00", "Z")
    timestamp_ms = int(now.timestamp()*1000)

    return timestamp_iso, timestamp_ms

In [18]:
def save_user_prompt(user_input, model, filename=user+"_input_history.csv"):

    file_exists = os.path.isfile(filename)
    
    with open(filename, mode='a', newline='') as file:
        writer = csv.writer(file)

        if not file_exists:
            writer.writerow(['timestamp_iso','timestamp_ms','model','user_input'])

        timestamp_iso, timestamp_ms = get_iso_time()
        
        writer.writerow([timestamp_iso, timestamp_ms, model, user_input])

In [19]:
save_user_prompt(user_input, model)